<a href="https://colab.research.google.com/github/iIsunnyIi/Funnel-Transformer/blob/main/Text_Classification_Funnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 9.4MB/s 
     |████████████████████████████████| 890kB 13.5MB/s 
     |████████████████████████████████| 133kB 23.5MB/s 
     |████████████████████████████████| 1.1MB 19.2MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 7.1MB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6310f147fcfa7ce53a33f8ddadbd5767cfffb6093e53d04ef7fd7719d705aa8a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.38 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.7MB/s 
     |████████████████████████████████| 2.9MB 26.2MB/s 


In [ ]:
!nvidia-smi

Thu Dec 17 01:47:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
>>> from transformers import FunnelTokenizer, FunnelForSequenceClassification
>>> import torch

In [ ]:
dataset_path = "/content/drive/MyDrive/nlu-benchmark-master/2017-06-custom-intent-engines/"

In [ ]:
dataset = pd.DataFrame(columns = ['utterance', 'label'])
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open(dataset_path + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Class: {}, # utterances: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
dataset.tail()

Class: AddToPlaylist, # utterances: 300
Class: BookRestaurant, # utterances: 300
Class: GetWeather, # utterances: 300
Class: PlayMusic, # utterances: 300
Class: RateBook, # utterances: 300
Class: SearchCreativeWork, # utterances: 300
Class: SearchScreeningEvent, # utterances: 300


,utterance,label
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent


In [ ]:
label_to_ix = {}
for label in dataset.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

In [ ]:
from transformers import FunnelModel, RobertaTokenizer
from transformers import FunnelForSequenceClassification, FunnelConfig

In [ ]:
config = FunnelConfig.from_pretrained('funnel-transformer/small-base')
config.num_labels = len(list(label_to_ix.values()))
config

FunnelConfig {
  "activation_dropout": 0.0,
  "architectures": [
    "FunnelBaseModel"
  ],
  "attention_dropout": 0.1,
  "attention_type": "relative_shift",
  "block_repeats": [
    1,
    1,
    1
  ],
  "block_sizes": [
    4,
    4,
    4
  ],
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "hidden_act": "gelu_new",
  "hidden_dropout": 0.1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.1,
  "initializer_std": null,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-09,
  "max_position_embeddings": 512,
  "model_type": "funnel",
  "n_head": 12,
  "num_decoder_layers": 2,
  "pool_q_only": true,
  "pooling_type": "mean",
  "rel_attn_type": "factorized",
  "separate_cls": true,
  "truncate_seq": true,
  "type_vocab_size": 3,
 

In [ ]:
tokenizer = FunnelTokenizer.from_pretrained('funnel-transformer/small-base')
model = FunnelForSequenceClassification(config)

In [ ]:
def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [ ]:
msg = "My dog is cute!"
prepare_features(msg)

(tensor([[  101,  2026,  3899,  2003, 10140,   999,   102]]),
 [1, 1, 1, 1, 1, 1, 1])

In [ ]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        utterance = self.data.utterance[index]
        label = self.data.label[index]
        X, _  = prepare_features(utterance)
        y = label_to_ix[self.data.label[index]]
        return X, y
    
    def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
train_dataset=dataset.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)

In [ ]:
print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2100, 2)
TRAIN Dataset: (1680, 2)
TEST Dataset: (420, 2)


In [ ]:
training_set = Intents(train_dataset)
testing_set = Intents(test_dataset)

In [ ]:
training_set.__getitem__(0)[0].shape

torch.Size([1, 8])

In [ ]:
model(training_set.__getitem__(0)[0])

SequenceClassifierOutput([('logits',
                           tensor([[ 0.3082, -0.6821,  0.3116, -1.0373,  1.0743,  0.0449,  0.9416]],
                                  grad_fn=<AddmmBackward>))])

In [ ]:
tokenizer = FunnelTokenizer.from_pretrained('funnel-transformer/small-base')
model = FunnelForSequenceClassification(config)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()

In [ ]:
# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1}

In [ ]:
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)

In [ ]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [ ]:
## Test Forward Pass
inp = training_set.__getitem__(0)[0].cuda()
output = model(inp)[0]
print(output.shape)

torch.Size([1, 7])


In [ ]:
torch.__version__

'1.7.0+cu101'

In [ ]:
max_epochs = 3
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        optimizer.zero_grad()
        sent = sent.squeeze(0)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
        output = model.forward(sent)[0]
        _, predicted = torch.max(output, 1)
        
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(0)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


EPOCH -- 0
Iteration: 0. Loss: 2.626661777496338. Accuracy: 15.238095238095237%
Iteration: 100. Loss: 0.4770510792732239. Accuracy: 35.714285714285715%
Iteration: 200. Loss: 0.38469088077545166. Accuracy: 71.19047619047619%
Iteration: 300. Loss: 0.05145356431603432. Accuracy: 85.47619047619048%
Iteration: 400. Loss: 0.03226786479353905. Accuracy: 74.52380952380952%
Iteration: 500. Loss: 0.04514886438846588. Accuracy: 73.33333333333333%
Iteration: 600. Loss: 0.0030939350835978985. Accuracy: 88.0952380952381%
Iteration: 700. Loss: 0.003863966092467308. Accuracy: 59.04761904761905%
Iteration: 800. Loss: 4.863243103027344. Accuracy: 93.0952380952381%
Iteration: 900. Loss: 0.004088615067303181. Accuracy: 84.28571428571429%
Iteration: 1000. Loss: 1.2305256128311157. Accuracy: 86.9047619047619%
Iteration: 1100. Loss: 0.031246880069375038. Accuracy: 91.66666666666667%
Iteration: 1200. Loss: 1.460211992263794. Accuracy: 94.76190476190476%
Iteration: 1300. Loss: 2.404374361038208. Accuracy: 92.3

In [ ]:
 torch.save(model.state_dict(), 'drive/My Drive/Datasets/Funnel_state_dict_'+ str(uuid4())+'.pth')

In [ ]:
model_path = '/content/drive/MyDrive/Datasets/Funnel_state_dict_1a0b7ecb-9385-4c94-b3dd-0f54b74212aa.pth'

In [ ]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [ ]:
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(label_to_ix.keys())[pred_label]
  return prediction

In [ ]:
label_to_ix.keys()

dict_keys(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'])

In [ ]:
get_reply("play radiohead song")

'PlayMusic'

In [ ]:
get_reply("it is rainy in Sao Paulo")

'GetWeather'

In [ ]:
get_reply("sun shinnes all day")

'RateBook'

In [ ]:
get_reply("low humidity, high altitude")

'RateBook'

In [ ]:
get_reply("Book tacos for me tonight")

'BookRestaurant'

In [ ]:
get_reply("Book a table for me tonight")

'BookRestaurant'

In [ ]:
get_reply("I want BBQ tonight")

'PlayMusic'